In [1]:
from census import Census
from us import states
import requests


In [2]:
c = Census("5663f37b7cd6f3f50ff75d2cc9963c453be6421d") # TODO: replace this key

lookup = {
    "Total!!Population of one race!!White": None,
    "Total!!Population of one race!!Black or African American": None,
    "Total!!Population of one race!!Asian": None,
    # TODO: what else?  Check all fields...
}
r = requests.get("https://api.census.gov/data/2010/dec/sf1/variables.json")
r.raise_for_status()
fields = r.json()["variables"]
for key, val in fields.items():
    desc = val["label"]
    if desc in lookup:
        lookup[desc] = key

print("Cencus codes:")
print(lookup)
rev = {v:k for k,v in lookup.items()}

def grab_zip(zipcode):
    #row = c.sf1.state_zipcode(list(lookup.values()), states.WI.fips, zipcode)[0]
    row = c.acs5.state_county_blockgroup(list(lookup.values()), states.WI.fips, zipcode)[0]
    keys = list(row.keys())
    for k in keys:
        if k in rev:
            new_key = rev[k].split("!!")[-1]
            val = row.pop(k)
            row[new_key] = val
    return row

for z in ["53717", "53703", "53715"]:
    print(z)
    print(grab_zip(z)) 


Cencus codes:
{'Total!!Population of one race!!White': 'PCT023003', 'Total!!Population of one race!!Black or African American': 'PCT023004', 'Total!!Population of one race!!Asian': 'PCT023006'}
53717
{'state': '55', 'zip code tabulation area (or part)': '53717', 'White': 9393.0, 'Black or African American': 575.0, 'Asian': 1497.0}
53703
{'state': '55', 'zip code tabulation area (or part)': '53703', 'White': 23818.0, 'Black or African American': 1289.0, 'Asian': 1703.0}
53715
{'state': '55', 'zip code tabulation area (or part)': '53715', 'White': 10070.0, 'Black or African American': 420.0, 'Asian': 1120.0}
